In [1]:
import matplotlib.pyplot as plt
# coding=utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import numpy as np
from keras.layers.core import Flatten

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Embedding, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dropout,LSTM,BatchNormalization,Conv1D,MaxPool1D,Reshape,Input,Conv2D
from tensorflow.keras.optimizers import SGD
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
import re
import torch
from keras.utils.vis_utils import plot_model
import math 
import os
import scipy.io as io


In [2]:
dataset = pd.read_csv('./Pistachio_Dataset/Pistachio_28_Features_Dataset/Pistachio_28_Features_Dataset.csv') 
cols = ['Class','Area', 'Perimeter', 'Major_Axis', 'Minor_Axis', 'Eccentricity',
        'Eqdiasq', 'Solidity', 'Convex_Area', 'Extent', 'Aspect_Ratio',
        'Roundness', 'Compactness', 'Shapefactor_1', 'Shapefactor_2',
        'Shapefactor_3', 'Shapefactor_4', 'Mean_RR', 'Mean_RG', 'Mean_RB',
        'StdDev_RR', 'StdDev_RG', 'StdDev_RB', 'Skew_RR', 'Skew_RG', 'Skew_RB',
        'Kurtosis_RR', 'Kurtosis_RG', 'Kurtosis_RB']
dataset = shuffle(dataset)
Lisclass = dataset['Class'].values
LisClassNp = []
for i in range(len(Lisclass)):
    if (Lisclass[i] == 'Kirmizi_Pistachio'):
        LisClassNp.append([0,1])
    else:
        LisClassNp.append([1,0])
Y = np.array(LisClassNp)
dataset = dataset.drop(columns='Class')
X = dataset.values
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)
#拆分训练集和测试集
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, random_state=40)
X_train = X_train.reshape(X_train.shape[0],1,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])

X_train = tf.cast(X_train, dtype='float32')
X_test = tf.cast(X_test, dtype='float32')
Y_train = tf.cast(Y_train, dtype='float32')
Y_test = tf.cast(Y_test, dtype='float32')

In [3]:
batch_size = 64 
sequence_lenth = 1 #每个句子的长度
input_size = 28    #每个时序的单独向量，代表每个字的含义
output_size = 128
x = X_train
#x = tf.random.uniform((batch_size,sequence_lenth,input_size))

In [4]:
#LSTM'S INOPUT: [batch_size,sequence_length,output_size]
#LSTM'S OUTPUT1: [batch_size,sequence_length,output_size]
#       OUTPUT2:[batch_size,output_size]

In [5]:
class CustomLSTM(tf.keras.layers.Layer):
    '''
    #LSTM'S INOPUT: [batch_size,sequence_length,output_size]
    #LSTM'S OUTPUT1: [batch_size,sequence_length,output_size]
    #       OUTPUT2:[batch_size,output_size]
    '''
    
    def __init__(self,output_size,return_sequences=False):
        super(CustomLSTM,self).__init__()
        self.output_size = output_size
        self.return_sequences = return_sequences
    def build(self,input_shape):
        super(CustomLSTM,self).build(input_shape)
        input_size = int(input_shape[-1])
        #self.wf = self.add_weight('wf', shape=(input_size,self.output_size),initializer='random_normal',trainable=True)
        self.wi = self.add_weight('wi', shape=(input_size,self.output_size),initializer='random_normal',trainable=True)
        self.wo = self.add_weight('wo', shape=(input_size,self.output_size),initializer='random_normal',trainable=True)
        self.wc = self.add_weight('wc', shape=(input_size,self.output_size),initializer='random_normal',trainable=True)

        #self.uf = self.add_weight('uf', shape=(self.output_size,self.output_size),initializer='random_normal',trainable=True)
        #self.ui = self.add_weight('ui', shape=(self.output_size,self.output_size),initializer='random_normal',trainable=True)
        #self.uo = self.add_weight('uo', shape=(self.output_size,self.output_size),initializer='random_normal',trainable=True)
        #self.uc = self.add_weight('uc', shape=(self.output_size,self.output_size),initializer='random_normal',trainable=True)

        #self.bf = self.add_weight('bf', shape=(1,self.output_size),initializer='random_normal',trainable=True)
        self.bi = self.add_weight('bi', shape=(1,self.output_size),initializer='random_normal',trainable=True)
        self.bo = self.add_weight('bo', shape=(1,self.output_size),initializer='random_normal',trainable=True)
        self.bc = self.add_weight('bc', shape=(1,self.output_size),initializer='random_normal',trainable=True)
    def call(self, x):
        Para = []
        sequence_outputs = []
        for i in range(sequence_lenth):
            if i == 0:
                xt = x[:, 0 ,:]
                it = tf.sigmoid(tf.matmul(xt,self.wi) + self.bi)
                ot = tf.sigmoid(tf.matmul(xt,self.wo) + self.bo)
                cht = tf.tanh(tf.matmul(xt,self.wc) + self.bc)
                ct = it * cht
                ht = ot * tf.tanh(ct)
                # para1 = para1.numpy()
                # para2 = para2.numpy()
                # para3 = para3.numpy()
                # para4 = para4.numpy()
                # ct = ct.numpy()
                # print(para1)
                # np.savetxt('para1.txt',para1)
                # np.savetxt('para2.txt',para2)
                # np.savetxt('para3.txt',para3)
                # np.savetxt('para4.txt',para4)
                # np.savetxt('ct.txt',ct)
                #io.savemat('save.mat',{'result1':result1})
            else:
                print('$$$$$$$$$$$$$$$$$$$$','this is else')
                # xt = x[:, i ,:]
                # para1 = tf.matmul(xt,self.wf) + tf.matmul(ht,self.uf) + self.bf
                # para2 = tf.matmul(xt,self.wi) + tf.matmul(ht,self.ui) + self.bi
                # para3 = tf.matmul(xt,self.wo) + tf.matmul(ht,self.uo) + self.bo
                # para4 = tf.matmul(xt,self.wc) + tf.matmul(ht,self.uc) + self.bc
                # ft = tf.sigmoid(para1)
                # it = tf.sigmoid(para2)
                # ot = tf.sigmoid(para3)
                # cht = tf.tanh(para4)
                # ct = ft * ct + it * cht
                # ht = ot * tf.tanh(ct)
                #Para.append([para1,para2,para3,para4,ct])
                #Note.write(f'para1:,{para1}\npara2:{para2}\npara3:{para3}\npara4:{para4}\nct:{ct}')
            sequence_outputs.append(ht)
        
        #Note.write(str(Para))
        #Note.close()
        sequence_outputs = tf.stack(sequence_outputs)
        sequence_outputs = tf.transpose(sequence_outputs,(1, 0, 2))
        if self.return_sequences:
            return sequence_outputs
        return sequence_outputs[:, -1 ,:]

In [6]:
class my_model(Model):
    def __init__(self, input_shape, **kwargs):
        super().__init__(**kwargs) 
        
        the_units = 128
        self.CNN   = Conv1D(the_units,1,padding="valid",activation="relu")
        self.POLL  = MaxPool1D(1)
        self.FLATT = Flatten()
        self.LSTM0 = CustomLSTM(output_size=the_units,return_sequences=True)
                        #kernel_initializer=tf.keras.initializers.glorot_normal(seed=2),bias_initializer=tf.keras.initializers.Zeros())
        self.LSTM1 = CustomLSTM(output_size=the_units,return_sequences=True)
        self.LSTM2 = CustomLSTM(output_size=the_units)
        self.BAT   = BatchNormalization()
        self.DROP  = Dropout(rate=0.2)
        self.DENS  = Dense(8, activation='relu')
        self.OUT   = Dense(2, activation='softmax')
        self.resha = Reshape((1, 28))
        
        # 模型的参数
        self.input_layer = tf.keras.layers.Input(input_shape)
        self.out = self.call(self.input_layer)
        
        super().__init__( inputs=self.input_layer,outputs=self.out, **kwargs)
    
    # 初始化模型的参数
    def build(self):
        self._is_graph_network = True
        self._init_graph_network(inputs=self.input_layer,outputs=self.out)
    
    def call(self, inputdata, **kwargs):
                
        #x1 = self.LSTM0(inputdata)
        print('inputdata',inputdata)
        x1 = self.CNN(inputdata)
        #x1 = self.LSTM0(inputdata)
        x1 = self.POLL(x1)
        c = self.DROP(x1)
        #c = self.BAT(x1)

        c = self.LSTM1(c)
        c = self.DROP(c)
        c = self.BAT(c)
        
        c = self.LSTM1(c)
        c = self.DROP(c)
        c = self.BAT(c)
        
        # # d = self.LSTM2(c)
        # # d = self.BAT(d)
        # c = self.LSTM1(c)
        # c = self.DROP(c)
        # c = self.BAT(c)
                        
        d = self.DENS(c)
        d = self.DROP(d)
        
        d = self.FLATT(d)
        out = self.OUT(d)
        return out
    # AFAIK: The most convenient method to print model.summary() 
    # similar to the sequential or functional API like.
    def build_graph(self):
        #x = Input(shape=(dim))
        x = Input()
        return Model(inputs=[x], outputs=self.call(x))

mymodel2 = my_model((1,28), name='MyModelMLP')
# mymodel2(X_train)
mymodel2.summary()


inputdata KerasTensor(type_spec=TensorSpec(shape=(None, 1, 28), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
Model: "MyModelMLP"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1, 28)]      0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 1, 128)       3712        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 1, 128)       0           conv1d[0][0]                     
__________________________________________________________________________________________________
dropout (Dropout)              

In [7]:
Epoch = 100  # 模型迭代的次数
Batch_Size = 64  # 批量训练的样本的个数
Out_Class = 2  # 输出的类别的个数,0-9共10类
# 模型编译
#mYMODEL = my_model((1,28),name="mYMODEL")
#mYMODEL(X_train)
mymodel2.compile(loss='binary_crossentropy',#categorical_crossentropy', binary_crossentropy
                optimizer='adam', metrics=['categorical_accuracy'])  

# train_label_cate = tf.keras.utils.to_categorical(train_labels, 10)
# testlabels = tf.keras.utils.to_categorical(test_labels, 10)
checkpoint_path = "./models/cp-{categorical_accuracy:.5f}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 创建一个回调，保证验证数据集准确率最大
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only=True,
                                                monitor='categorical_accuracy',
                                                mode='max',
                                                verbose=2,
                                                save_best_only=True)
#                                              filepath=filepath,
#                                             save_weights_only=False,
#                                             monitor='categorical_accuracy',
#                                             mode='max',
#                                             save_best_only=True)

# 动态更改学习率：在模型的回调中使用
def scheduler(epoch):  # 根据epoch动态更改学习率的参数
    if epoch < 10:
        return 0.13
    else:
        return 0.13 * tf.math.exp(0.1 * (10 - epoch))
    
lr_back = tf.keras.callbacks.LearningRateScheduler(scheduler)


In [8]:
# Note = open('LSTM-para.txt',mode='w')
# Note.close()
mymodel2.fit(X_train,Y_train, batch_size=Batch_Size, 
            epochs=Epoch, verbose=2, validation_split=0.1, 
            callbacks=cp_callback)
            #callbacks=[cp_callback, lr_back])


Epoch 1/100
inputdata Tensor("IteratorGetNext:0", shape=(None, 1, 28), dtype=float32)
inputdata Tensor("IteratorGetNext:0", shape=(None, 1, 28), dtype=float32)


UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node MyModelMLP/conv1d/conv1d (defined at C:\Users\20987\AppData\Local\Temp\ipykernel_14988\3388358910.py:34) ]] [Op:__inference_train_function_2097]

Function call stack:
train_function


In [ ]:
# 评估模型,按照模型最后的参数计算
test_loss, test_acc = mymodel2.evaluate(X_test, Y_test)

print('测试数据集成本：{:.8f},准确率{:.8f}%%'. format(test_loss, 100*test_acc))


best_para = tf.train.latest_checkpoint(checkpoint_dir)
print('最优的参数文件：', best_para)
#best_para = './models\cp-0.92987.ckpt'
mymodel2.load_weights(best_para)

predict_loss, predict_acc = mymodel2.evaluate(X_test, Y_test)
print('使用训练后的参数','成本:', predict_loss, '准确率', predict_acc)

使用其他数据集去做验证
可行性分析，会拿几种进行对比
数据分析与处理
在输入到模型之前，有一个优化器，该优化器是对指标进行排序的，是一个固定的模式
顺序优化的优化器。基于优化器，针对大数据，使用LSTM对所有指标进行分类（重要性分类），对几类的数据做相应优化，
越不重要越先输入还是越重要越先输入。重要性体现在什么地方。

优化器-> LSTM（对内部超参数进行优化）
找到其他LSTM的改进，进行实验。最近三年来新的LSTM改进。

改进——>.0本身的改进
    -->基本原理的改进，逻辑门的改进

In [ ]:
# from pickletools import optimize


# model = tf.keras.Sequential([
#     CustomLSTM(output_size=128),
#     tf.keras.layers.Dense(2, activation='softmax')
# ])
# model.compile(loss='binary_crossentropy',#categorical_crossentropy', binary_crossentropy
#                 optimizer='adam', metrics=['categorical_accuracy'])  

In [ ]:
# model.fit(X_train,Y_train,batch_size=64)